In [1]:
!rm -rf ctcdecode
%cd /content
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .
%cd /content

/content
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1102 (delta 16), reused 28 (delta 13), pack-reused 1063
Receiving objects: 100% (1102/1102), 780.91 KiB | 12.59 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14057, done.        
remote: Counting objects: 100% (370/370), done.        
remote: Compressing objects: 100% (299/299), done.        
remote: Total 14057 

In [2]:
!rm -rf Wav2Vec2_PyCTCDecode
!git clone --recursive https://github.com/patrickvonplaten/Wav2Vec2_PyCTCDecode.git

Cloning into 'Wav2Vec2_PyCTCDecode'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 43 (delta 18), reused 15 (delta 5), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [3]:
!pip install -q torchaudio
!pip install -q transformers>=4.12.0
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic
!pip install -q ctc-segmentation
!pip install -q pyctcdecode
!pip install -q https://github.com/kpu/kenlm/archive/master.zip

     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 52.5 MB/s 
     |████████████████████████████████| 132 kB 66.1 MB/s 
     |████████████████████████████████| 243 kB 56.1 MB/s 
     |████████████████████████████████| 192 kB 62.8 MB/s 
     |████████████████████████████████| 271 kB 36.8 MB/s 
     |████████████████████████████████| 160 kB 50.2 MB/s 
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 126 kB 5.1 MB/s 
     |████████████████████████████████| 71 kB 3.8 MB/s 
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 377 kB 10.0 MB/s 
     \ 541 kB 1.9 MB/s


In [4]:
%cd Wav2Vec2_PyCTCDecode

/content/Wav2Vec2_PyCTCDecode


In [5]:
import os
import re
import sys
import json
import torch
import jiwer
import kenlm
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
import ctc_segmentation
import tensorflow as tf
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
# from trainer import CTCTrainer
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from pyctcdecode import build_ctcdecoder
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from sklearn.metrics import accuracy_score
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset,load_metric
from keras.preprocessing.sequence import pad_sequences
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

In [6]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/dataset_wav.zip .

In [7]:
if os.path.exists('dataset_wav'):
  if len(os.listdir('dataset_wav')) == 0:
    os.system('unzip dataset_wav.zip')
else:
  os.system('unzip dataset_wav.zip')

In [8]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'

In [9]:
dataset_wav_folder = 'dataset_wav'

In [10]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_wav_folder}/{file_name}'))
  metadata = metadata[metadata['File Exist'] == True]
  return metadata

In [11]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
test_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


In [12]:
metadata_14_bahr = metadata[[bahr not in ('المضارع','المقتضب') for bahr in metadata['Bahr']]]
metadata_14_bahr

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,1,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...,True
1,2,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن ا...,True
2,3,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام,True
3,4,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام,True
4,5,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام,True
...,...,...,...,...,...,...,...,...,...,...,...
3681,3731,P-3731.mp3,وَلِّ السَلاطينَ مَن تَوَلّاها وَاِلجَأ إ...,المنسرح,-,45.mp3,45.mp3,NaN,NaN,ول السلاطين من تولاها والجأ إليه تكن حدياها,True
3682,3732,P-3732.mp3,وَلا تَغُرَّنَّكَ الإِمارَةُ في غَيرِ أَم...,المنسرح,-,46.mp3,46.mp3,NaN,NaN,ولا تغرنك الإمارة في غير أمير وإن بها باهى,True
3683,3733,P-3733.mp3,فَإِنَّما المَلكُ رَبُّ مَملَكَةٍ قَد فَغ...,المنسرح,-,47.mp3,47.mp3,NaN,NaN,فإنما الملك رب مملكة قد فغم الخافقين سرياها,True
3684,3734,P-3734.mp3,مُبتَسِمٌ وَالوُجوهُ عابِسَةٌ سِلمُ العِد...,المنسرح,-,48.mp3,48.mp3,NaN,NaN,مبتسم والوجوه عابسة سلم العدى عنده كهيجاها,True


In [13]:
test_metadata_14_bahr = test_metadata[[bahr not in ('المضارع','المقتضب') for bahr in test_metadata['Bahr']]]
test_metadata_14_bahr

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


## Wrapper Classes

In [14]:
class Transcriber:
    def __init__(self, model_path, processor_path="bakrianoo/sinai-voice-ar-stt"):
        self.processor = Wav2Vec2Processor.from_pretrained(processor_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path).eval()
        self.model.to("cuda")

    def transcribe(self, file_path, with_ctc=True):
        speech, sampling_rate = librosa.load(file_path, sr=16_000)
        inputs = self.processor(
            speech, sampling_rate=16_000, return_tensors="pt", padding=True
        )
        with torch.no_grad():
          logits = self.model(inputs.input_values.to('cuda')).logits
          for logit in logits:
            logit[logit==-100] = self.processor.tokenizer.pad_token_id
          predicted = torch.argmax(logits, dim=-1)
        # predicted[predicted == -100] = self.processor.tokenizer.pad_token_id
        predicted_tokenized = self.processor.tokenizer.batch_decode(predicted)
        clean_predicted_tokenized = [araby.strip_tashkeel(araby.strip_tatweel(predicted_tokenized[0]))]
        return predicted_tokenized

In [15]:
lm_path = "/content/drive/MyDrive/KFUPM-Master/ICS606/Models/baits_dataset_kenlm_model.arpa"

In [16]:
!python fix_lm.py --path_to_ngram /content/drive/MyDrive/KFUPM-Master/ICS606/Models/baits_dataset_kenlm_model.arpa --path_to_fixed fixed_baits_dataset_kenlm.arpa

In [17]:
fixed_lm_path = 'fixed_baits_dataset_kenlm.arpa'

In [18]:
class CTCTranscriber:
    def __init__(self, model_path, lm_model_path='fixed_baits_dataset_kenlm.arpa', processor_path="bakrianoo/sinai-voice-ar-stt"):
        self.processor = Wav2Vec2Processor.from_pretrained(processor_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path).eval()
        self.model.to("cuda")
        self.lm_model_path = lm_model_path
        self.decoder = build_ctcdecoder(
            self.vocab,
            self.lm_model_path,
            alpha=0.5,
        )

    @property
    def vocab(self):
      vocab_dict = self.processor.tokenizer.get_vocab()
      sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())
      vocab = [token for _,token in sort_vocab]
      return vocab

    def transcribe(self, file_path, with_ctc=True):
        speech, sampling_rate = librosa.load(file_path, sr=16_000)
        inputs = self.processor(
            speech, sampling_rate=16_000, return_tensors="pt", padding=True
        )
        with torch.no_grad():
          logits = self.model(inputs.input_values.to('cuda')).logits
          for logit in logits:
            logit[logit==-100] = self.processor.tokenizer.pad_token_id
          decoded = [araby.strip_tashkeel(self.decoder.decode(logits.cpu().numpy()[0], beam_width=1000))]
        return decoded

In [19]:
char2idx = {
    " ": 1,
    "#": 2,
    "ء": 3,
    "آ": 4,
    "أ": 5,
    "ؤ": 6,
    "إ": 7,
    "ئ": 8,
    "ا": 9,
    "ب": 10,
    "ة": 11,
    "ت": 12,
    "ث": 13,
    "ج": 14,
    "ح": 15,
    "خ": 16,
    "د": 17,
    "ذ": 18,
    "ر": 19,
    "ز": 20,
    "س": 21,
    "ش": 22,
    "ص": 23,
    "ض": 24,
    "ط": 25,
    "ظ": 26,
    "ع": 27,
    "غ": 28,
    "ف": 29,
    "ق": 30,
    "ك": 31,
    "ل": 32,
    "م": 33,
    "ن": 34,
    "ه": 35,
    "و": 36,
    "ى": 37,
    "ي": 38,
}
meters_mapping = {
    0: 4,
    1: 6,
    2: 8,
    3: 7,
    4: 13,
    5: 10,
    6: 9,
    7: 3,
    8: 0,
    9: 1,
    10: 5,
    11: 15,
    12: 14,
    13: 2,
}
meter_class_to_label = {v:k for k,v in {
    "البسيط": 0,
    "الخفيف": 1,
    "الرجز": 2,
    "الرمل": 3,
    "السريع": 4,
    "الطويل": 5,
    "الكامل": 6,
    "المتدارك": 7,
    "المتقارب": 8,
    "المجتث": 9,
    "المديد": 10,
    "المضارع": 11,
    "المقتضب": 12,
    "المنسرح": 13,
    "الهزج": 14,
    "الوافر": 15,
}.items()}

meter_label_to_class = {
    "البسيط": 0,
    "الخفيف": 1,
    "الرجز": 2,
    "الرمل": 3,
    "السريع": 4,
    "الطويل": 5,
    "الكامل": 6,
    "المتدارك": 7,
    "المتقارب": 8,
    "المجتث": 9,
    "المديد": 10,
    "المضارع": 11,
    "المقتضب": 12,
    "المنسرح": 13,
    "الهزج": 14,
    "الوافر": 15,
}

In [20]:
resamplers = {  # all three sampling rates exist in test split
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    32000: torchaudio.transforms.Resample(32000, 16000),
}

def get_resampled_speech(file_path):
    original_speech, sampling_rate = torchaudio.load(file_path)
    if sampling_rate in resamplers:
        speech = resamplers[sampling_rate](original_speech[0]).squeeze().numpy()
    else:
        speech = resamplers[48000](original_speech[0]).squeeze().numpy()
    return speech

def put_hash_after_middle_word(sentence):
    middle_char = len(sentence) // 2
    closest_space_index = middle_char
    while True:
        try:
            if sentence[closest_space_index] == " ":
                break
            else:
                closest_space_index += 1
        except:
            closest_space_index = len(sentence) - 1
            break
    sentence = sentence[:closest_space_index] + " #" + sentence[closest_space_index:]
    return sentence

In [21]:
class TextClassifier:
    def __init__(self, transcriber, model_path='/content/drive/MyDrive/KFUPM-Master/ICS606/Models/arabic_poetry_text_classification_model.h5'):
        self.model = tf.keras.models.load_model(model_path)
        self.transcriber = transcriber

    def classify(self, file_path):
        predicted_text = self.transcriber.transcribe(file_path)[0]
        predicted_text = araby.strip_tashkeel(predicted_text)
        predicted_text = put_hash_after_middle_word(predicted_text)
        sequence = [char2idx[char] for char in predicted_text if char2idx.get(char)]
        sequence = pad_sequences([sequence], maxlen=100, padding="post", value=0)
        predictions = self.model.predict(sequence)[0]
        label = meters_mapping[np.argmax(predictions, 0).astype("int")]
        return meter_class_to_label[label]

In [22]:
MODEL_ID = "bakrianoo/sinai-voice-ar-stt"
# processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
# model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

In [23]:
transcriber = Transcriber(model_path=MODEL_ID)
classifier = TextClassifier(transcriber=transcriber)

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [24]:
ctc_transcriber = CTCTranscriber(model_path=MODEL_ID)
ctc_classifier = TextClassifier(transcriber=ctc_transcriber)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


In [25]:
dataset = Dataset.from_pandas(metadata_14_bahr)

#WER and CER

## Transcribtion Without LM

In [26]:
wer = load_metric('wer')
cer = load_metric('cer')

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [27]:
def predict_text(example):
  example['predicted_text'] = transcriber.transcribe(f'{dataset_wav_folder}/{example["Utterance name"]}')
  return example

In [ ]:
dataset = dataset.map(predict_text)

  0%|          | 0/3416 [00:00<?, ?ex/s]

In [ ]:
print(wer.compute(predictions=dataset["predicted_text"], references=dataset["Clean Script"]))
print(cer.compute(predictions=dataset["predicted_text"], references=dataset["Clean Script"]))

0.44853723833177433
0.15953495258756656


## Transcribtion With LM

In [ ]:
def predict_text(example):
  example['predicted_text'] = ctc_transcriber.transcribe(f'{dataset_wav_folder}/{example["Utterance name"]}')
  return example

In [ ]:
dataset = dataset.map(predict_text)

In [ ]:
print(wer.compute(predictions=dataset["predicted_text"], references=dataset["Clean Script"]))
print(cer.compute(predictions=dataset["predicted_text"], references=dataset["Clean Script"]))

# Classification

## Without CTC Decoding

In [ ]:
def predict(example):
  example['predicted'] = classifier.classify(f'{dataset_wav_folder}/{example["Utterance name"]}')
  return example

In [ ]:
classifier.classify(f"{dataset_wav_folder}/{dataset[30]['Utterance name']}"),{dataset[30]['Bahr']}

In [ ]:
dataset = dataset.map(predict)

In [ ]:
def compute_metrics(preds,labels):
    print(labels)
    print(preds)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds,digits=4)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    print('clasification report',report)
    print('accuracy',acc)

In [ ]:
compute_metrics(preds=[example['predicted'] for example in dataset],labels=[example['Bahr'] for example in dataset])

## With CTC Decoding

In [ ]:
def predict(example):
  example['predicted'] = ctc_classifier.classify(f'{dataset_wav_folder}/{example["Utterance name"]}')
  return example

In [ ]:
dataset = dataset.map(predict)

In [ ]:
def compute_metrics(preds,labels):
    print(labels)
    print(preds)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    print('clasification report',report)
    print('accuracy',acc)

In [ ]:
compute_metrics(preds=[example['predicted'] for example in dataset],labels=[example['Bahr'] for example in dataset])

In [ ]:
index = 100 #@param {type:"slider", min:0, max:359, step:1}
print(ctc_transcriber.transcribe(f'{dataset_wav_folder}/{dataset[index]["Utterance name"]}'))
print(transcriber.transcribe(f'{dataset_wav_folder}/{dataset[index]["Utterance name"]}'))
print(f'{dataset[index]["Clean Script"]}')